# ViT Playground
copyright 2023, Denis Rothman

Installaing gradio, transformers, torch, numpy and pillow

In [1]:
!pip install --upgrade gradio transformers torch numpy pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 123.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0

Restart the runtime if requested.

In [1]:
import torch
import numpy as np
from PIL import Image
from transformers import AutoModelForImageClassification
import gradio as gr

In [2]:
# Load the model
model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")

In [3]:
import pandas as pd

labels = model.config.id2label

# convert the dictionary to a dataframe
df_labels = pd.DataFrame(list(labels.items()), columns=['ID', 'Label'])

# print the dataframe
df_labels

,ID,Label
0,0,"tench, Tinca tinca"
1,1,"goldfish, Carassius auratus"
2,2,"great white shark, white shark, man-eater, man..."
3,3,"tiger shark, Galeocerdo cuvieri"
4,4,"hammerhead, hammerhead shark"
...,...,...
995,995,earthstar
996,996,"hen-of-the-woods, hen of the woods, Polyporus ..."
997,997,bolete
998,998,"ear, spike, capitulum"


In [4]:
def classify_image(image: Image.Image):
    try:
        # Convert the image to a numpy array and normalize
        image = np.array(image) / 255.0
        # Adjust the image shape (C, H, W)
        image = np.transpose(image, (2, 0, 1))
        # Convert the image to a tensor
        image = torch.tensor(image).unsqueeze(0).float()
        # Classify the image
        predictions = model(image)
        label_ids = predictions.logits.argmax(axis=1)
        labels = model.config.id2label
        lid = int(label_ids[0])  # From tensor to int
        return labels[lid]
    except Exception as e:
        print(f"Exception: {e}")
        import traceback
        traceback.print_exc()
        return str(e)

In [5]:
# Define Gradio interface
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.inputs.Image(shape=(224, 224)),
    outputs="text"
)

iface.launch()

<ipython-input-5-347eca1362d5>:4: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Image(shape=(224, 224)),
<ipython-input-5-347eca1362d5>:4: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Image(shape=(224, 224)),


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>